# Cars price prediction

## I. Data import

In [326]:
import pandas as pd
import numpy as np
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('ScrapyParser/kolesa_cars.csv')
df

,manufacturer,model,price,year,body,engine_volume,fuel_type,transmission,city,date
0,Renault,Megane,1 800 000,2004,седан,1.4,бензин,механика,Актобе,29 марта
1,ВАЗ (Lada),2131 (5-ти дверный),1 055 000,2004,внедорожник,1.8,бензин,механика,Актобе,29 марта
2,Chevrolet,Orlando,8 000 000,2013,минивэн,1.8,бензин,автомат,Актобе,29 марта
3,Kia,Cee’d,6 300 000,2013,хэтчбек,1.6,бензин,автомат,Актобе,29 марта
4,Toyota,Camry,17 000 000,2018,седан,2.5,бензин,автомат,Актобе,29 марта
...,...,...,...,...,...,...,...,...,...,...
131475,Toyota,Camry,6 500 000,2006,седан,2.4,бензин,автомат,Актобе,29 марта
131476,BMW,320,3 600 000,2002,седан,2.2,бензин,автомат,Актобе,29 марта
131477,ГАЗ,ГАЗель,6 700 000,2013,пикап,2.9,газ-бензин,механика,Актобе,29 марта
131478,ВАЗ (Lada),Priora 2170 (седан),3 200 000,2014,седан,1.6,бензин,механика,Актобе,29 марта


## II. Data cleaning

#### Remove duplicates
Due to the inherent nature of the repetition of ads and due to the technical specifics of the parser, there are duplicates in the dataset that need to be removed.

In [327]:
print(f'Total amount of rows: {len(df)}\n'
      f'Total amount of duplicates: {len(df)-len(df.drop_duplicates())}')
df = df.drop_duplicates()
print(f'New amount: {len(df)}')

Total amount of rows: 131480
Total amount of duplicates: 15759
New amount: 115721


#### Drop useless columns
The parsing occurred on March 29 and, accordingly, the date in the column is only 7 days. This data is useless.

In [328]:
print(f"All days in dataset: {df['date'].unique()}")
df.drop('date', inplace=True, axis=1)

All days in dataset: ['29 марта' '25 марта' '23 марта' '24 марта' '26 марта' '28 марта'
 '27 марта' '30 марта']


#### Drop garbage rows
Parser may set wrong values for some columns, let's check their status.

In [329]:
print(f'Manufacturer: OK\n{df.manufacturer.unique()}\n\n'
      f'Model: OK\n{df.model.unique()}\n\n'
      f'Price: OK\n{df.price.unique()}\n\n'
      f'Year: OK\n{df.year.unique()}\n\n'
      f'Body: BAD\n{df.body.unique()}\n\n'
      f'Engine volume: BAD\n{df.engine_volume.unique()}\n\n'
      f'Fuel type: BAD\n{df.fuel_type.unique()}\n\n'
      f'Transmission: BAD\n{df.transmission.unique()}\n\n'
      f'City: Ok\n{df.city.unique()}\n\n')

Manufacturer: OK
['Renault' 'ВАЗ (Lada)' 'Chevrolet' 'Kia' 'Toyota' 'Nissan' 'Opel'
 'Hyundai' 'ГАЗ' 'Daewoo' 'Skoda' 'Mitsubishi' 'Volkswagen' 'Lexus'
 'Geely' 'УАЗ' 'SsangYong' 'Mercedes-Benz' 'Ford' 'BMW' 'Audi' 'Changan'
 'Subaru' 'FAW' 'Great Wall' 'Porsche' 'Mazda' 'Honda' 'Москвич'
 'Infiniti' 'Suzuki' 'Haval' 'Land Rover' 'Ravon' 'Dodge' 'Mini' 'Peugeot'
 'JAC' 'ЗАЗ' 'Derways' 'Foton' 'ИЖ' 'Fiat' 'Lincoln' 'Lifan' 'Jaguar'
 'Chery' 'Datsun' 'Daihatsu' 'Citroen' 'Chrysler' 'Volvo' 'Isuzu' 'Seat'
 'Cadillac' 'Jeep' 'GMC' 'Hummer' 'Alfa Romeo' 'MG' 'ЛуАЗ' 'DongFeng'
 'Acura' 'Pontiac' 'Shuanghuan' 'Bentley' 'Saab' 'ТагАЗ' 'Tianma' 'BYD'
 'Ретро-автомобили' 'Hafei' 'ЗиЛ' 'Lancia' 'ZX' 'Rover' 'ВИС' 'Haima'
 'Smart' 'Maserati' 'Wuling' 'Genesis' 'Jin' 'GAC' 'Tesla' 'РАФ' 'Gonow'
 'Dacia' 'Iran Khodro' 'Mercedes-Maybach' 'Rolls-Royce' 'Buick' 'Scion'
 'Mercury' 'Aston Martin' 'BAIC' 'Maybach' 'Ferrari' 'Dayun' 'Lamborghini'
 'Puch' 'Huanghai' 'BAW' 'Mahindra' 'Tianye']

Model: OK
['M

As we can see engine_volume, body, fuel_type, transmission columns has some garbage rows, we will delete them.

In [330]:
df['engine_volume'] = df['engine_volume'].astype(str)
df['engine_volume'] = df['engine_volume'].apply(lambda x: None if x in ['газ', 'бензин', 'электричество'] else x)
df['body'] = df['body'].apply(lambda x: None if x.strip()=='л' else x)
df['fuel_type'] = df['fuel_type'].apply(lambda x: None if x in ['КПП Автомат', 'КПП механика', 'правый руль'] else x)
df['transmission'] = df['transmission'].apply(lambda x: x if x in ['механика', 'автомат', 'вариатор', 'робот'] else None)
df.dropna(inplace=True)

Now these columns are good to go.

In [331]:
print(f'Body: OK\n{df.body.unique()}\n\n'
      f'Engine volume: OK\n{df.engine_volume.unique()}\n\n'
      f'Fuel type: OK\n{df.fuel_type.unique()}\n\n'
      f'Transmission: OK\n{df.transmission.unique()}\n\n')

Body: OK
['седан' 'внедорожник' 'минивэн' 'хэтчбек' 'универсал' 'фургон'
 'кроссовер' 'лифтбек' 'пикап' 'микроавтобус' 'купе' 'родстер' 'микровэн'
 'лимузин' 'кабриолет' 'хардтоп' 'фастбек' 'тарга']

Engine volume: OK
['1.4' '1.8' '1.6' '2.5' '3' '4.5' '2' '1.5' '2.7' '2.9' '0.8' '2.4' '1.7'
 '4.6' '2.3' '3.5' '2.8' '4' '1.3' '4.7' '5.5' '2.2' '1.1' '3.6' '3.3'
 '3.2' '2.6' '2.1' '4.4' '5.7' '1.2' '1.9' '6' '3.4' '4.8' '5' '4.3' '5.6'
 '3.8' '1' '5.4' '4.2' '3.7' '3.1' '6.2' '6.3' '3.9' '5.3' '8.5' '0.7'
 '0.9' '6.5' '9.9' '7.7' '1.69' '0.6' '6.1' '5.9' '0.2' '4.9' '5.2' '8.3'
 '6.8' '9' '5.8' '0.1' '6.6' '7.4' '6.7' '8.2' '6.4' '4.497' '8.1' '2.99'
 '7.3' '2.01' '8' '2.0' '4.0' '3.0' '5.0']

Fuel type: OK
['бензин' 'газ-бензин' 'дизель' 'гибрид' 'газ' 'электричество']

Transmission: OK
['механика' 'автомат' 'вариатор' 'робот']




As the result of data cleaning:
Rows amount at the start: 131480
Rows amount after duplicates removed: 115721
Rows amount after garbage rows are removed: 109483

In [332]:
df

,manufacturer,model,price,year,body,engine_volume,fuel_type,transmission,city
0,Renault,Megane,1 800 000,2004,седан,1.4,бензин,механика,Актобе
1,ВАЗ (Lada),2131 (5-ти дверный),1 055 000,2004,внедорожник,1.8,бензин,механика,Актобе
2,Chevrolet,Orlando,8 000 000,2013,минивэн,1.8,бензин,автомат,Актобе
3,Kia,Cee’d,6 300 000,2013,хэтчбек,1.6,бензин,автомат,Актобе
4,Toyota,Camry,17 000 000,2018,седан,2.5,бензин,автомат,Актобе
...,...,...,...,...,...,...,...,...,...
131474,ВАЗ (Lada),Priora 2170 (седан),2 700 000,2013,седан,1.6,бензин,механика,Актобе
131476,BMW,320,3 600 000,2002,седан,2.2,бензин,автомат,Актобе
131477,ГАЗ,ГАЗель,6 700 000,2013,пикап,2.9,газ-бензин,механика,Актобе
131478,ВАЗ (Lada),Priora 2170 (седан),3 200 000,2014,седан,1.6,бензин,механика,Актобе


## III. Data preparation
Only "year" columns has appropriate type. So we will cast each of other columns.

In [333]:
df.dtypes

manufacturer     object
model            object
price            object
year              int64
body             object
engine_volume    object
fuel_type        object
transmission     object
city             object
dtype: object

We will give columns consisting of digits the correct types

In [334]:
df['price'] = df['price'].apply(lambda x: x.replace(' ', '')).astype(int)
df['engine_volume'] = df['engine_volume'].astype(float)

Transmission and fuel type has few categories, so we will turn them into binary columns

In [335]:
df = df.join(pd.get_dummies(df.transmission))
df = df.join(pd.get_dummies(df.fuel_type))

We don't want to overfit our model with many columns, so we can not just make dummy values from columns as manufacturer, model, body, city. We will drop them.

In [336]:
df_num = df.drop(['manufacturer', 'model', 'body', 'fuel_type', 'transmission', 'city'], axis=1)
df_num

,price,year,engine_volume,автомат,вариатор,механика,робот,бензин,газ,газ-бензин,гибрид,дизель,электричество
0,1800000,2004,1.4,0,0,1,0,1,0,0,0,0,0
1,1055000,2004,1.8,0,0,1,0,1,0,0,0,0,0
2,8000000,2013,1.8,1,0,0,0,1,0,0,0,0,0
3,6300000,2013,1.6,1,0,0,0,1,0,0,0,0,0
4,17000000,2018,2.5,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131474,2700000,2013,1.6,0,0,1,0,1,0,0,0,0,0
131476,3600000,2002,2.2,1,0,0,0,1,0,0,0,0,0
131477,6700000,2013,2.9,0,0,1,0,0,0,1,0,0,0
131478,3200000,2014,1.6,0,0,1,0,1,0,0,0,0,0


At the end, we have ready for regression, filled only with numeric values dataframe

In [337]:
df_num.dtypes

price              int64
year               int64
engine_volume    float64
автомат            uint8
вариатор           uint8
механика           uint8
робот              uint8
бензин             uint8
газ                uint8
газ-бензин         uint8
гибрид             uint8
дизель             uint8
электричество      uint8
dtype: object

## IV. Regression

In [338]:
y = df_num.price
df_num.drop(['price'], axis=1, inplace=True)
X = df_num
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)


In [340]:
model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)


print(mean_absolute_error(y_test,pred))
print(mean_squared_error(y_test,pred))

3184086.935042661
31893925417651.496
